In [1]:
import pandas as pd
import numpy as np
import re
import prepare

Use this to drop normalized columns if necessary:


    columns = []
    for i in df.columns:
        columns.append(i)
    columns = str(columns)
    normalized_columns = re.findall(r'(smart_\d+_normalized)', columns)
    df.drop(columns=normalized_columns, inplace=True)"""

In [2]:
df = pd.read_csv('hard_drives_smart_5.csv')

In [3]:
df.shape

(169073, 11)

In [4]:
df.head(10)

,Unnamed: 0,serial_number,model,capacity_bytes,max(failure),max(smart_9_raw),max(smart_5_raw),max(smart_187_raw),max(smart_188_raw),max(smart_197_raw),max(smart_198_raw)
0,0,PL1311LAG1SJAA,Hitachi HDS5C4040ALE630,4000787030016,0,43819.0,0.0,NaN,NaN,0.0,0.0
1,1,Z305KB36,ST4000DM000,4000787030016,0,31045.0,0.0,0.0,0.0,0.0,0.0
2,2,MJ0351YNG9MZXA,Hitachi HDS5C3030ALA630,3000592982016,0,41668.0,0.0,NaN,NaN,0.0,0.0
3,3,ZA11NHSN,ST8000DM002,8001563222016,0,26284.0,0.0,0.0,0.0,0.0,0.0
4,4,MJ1311YNG2ZSEA,Hitachi HDS5C3030ALA630,3000592982016,0,47994.0,0.0,NaN,NaN,0.0,0.0
5,5,Z305D5VF,ST4000DM000,4000787030016,0,31883.0,0.0,0.0,0.0,0.0,0.0
6,6,Z3015L2E,ST4000DM000,4000787030016,0,36768.0,0.0,0.0,0.0,0.0,0.0
7,7,PL1331LAHDYJYH,HGST HMS5C4040BLE640,4000787030016,0,30144.0,0.0,NaN,NaN,0.0,0.0
8,8,Z304LAHP,ST4000DM000,4000787030016,0,33885.0,0.0,0.0,0.0,0.0,0.0
9,9,Z302B0GK,ST4000DM000,4000787030016,0,36443.0,0.0,0.0,0.0,0.0,0.0


In [5]:
prepare.prepare(df)

,serial_number,manufacturer,model,capacity_gigabytes,failure,drive_age_in_years,reallocated_sectors_count,reported_uncorrectable_errors,command_timeout,current_pending_sector_count,uncorrectable_sector_count
0,PL1311LAG1SJAA,Hitachi,Hitachi HDS5C4040ALE630,4000.8,0,5.0,0.0,NaN,NaN,0.0,0.0
1,Z305KB36,Seagate,ST4000DM000,4000.8,0,3.5,0.0,0.0,0.0,0.0,0.0
2,MJ0351YNG9MZXA,Hitachi,Hitachi HDS5C3030ALA630,3000.6,0,4.8,0.0,NaN,NaN,0.0,0.0
3,ZA11NHSN,Seagate,ST8000DM002,8001.6,0,3.0,0.0,0.0,0.0,0.0,0.0
4,MJ1311YNG2ZSEA,Hitachi,Hitachi HDS5C3030ALA630,3000.6,0,5.5,0.0,NaN,NaN,0.0,0.0
5,Z305D5VF,Seagate,ST4000DM000,4000.8,0,3.6,0.0,0.0,0.0,0.0,0.0
6,Z3015L2E,Seagate,ST4000DM000,4000.8,0,4.2,0.0,0.0,0.0,0.0,0.0
7,PL1331LAHDYJYH,Hitachi,HGST HMS5C4040BLE640,4000.8,0,3.4,0.0,NaN,NaN,0.0,0.0
8,Z304LAHP,Seagate,ST4000DM000,4000.8,0,3.9,0.0,0.0,0.0,0.0,0.0
9,Z302B0GK,Seagate,ST4000DM000,4000.8,0,4.2,0.0,0.0,0.0,0.0,0.0


In [6]:
df.dtypes

serial_number                     object
model                             object
capacity_gigabytes               float64
failure                            int64
drive_age_in_years               float64
reallocated_sectors_count        float64
reported_uncorrectable_errors    float64
command_timeout                  float64
current_pending_sector_count     float64
uncorrectable_sector_count       float64
manufacturer                      object
dtype: object